In [22]:
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw 
from rdkit.Chem import rdmolops
from rdkit.Chem.Draw import IPythonConsole
import numpy as np

In [45]:
vyb = Chem.SDMolSupplier('sets/cox2_3d.sd')

ms = [x for x in vyb if x is not None]
len(ms)

467

In [46]:
def clean_mark(mark, n) :
    
    for i in range(n) :
        mark[i] = 0

In [47]:
def wave(a, ii, n, mark) :
    
    f = 1
    
    mark_num = 1

    # поджог
    mark[ii] = mark_num

    # пока не подожжем все вершины
    while 1 :
        if f == 0 :
            break

        f = 0

        for i in range (n) :
            if mark[i] == mark_num :
                for j in range (n) :
                    if a[i][j] == 1 and mark[j] == 0 :
                        mark[j] = mark_num + 1
    
        for i in range (n) :
            if mark[i] == 0 :
                f = 1

        mark_num = mark_num + 1

In [48]:
def init_distance_table_i(d, mark, i, n) :

    for j in range(n) :
        d[i][j] = mark[j] - 1

In [49]:
def init_distance_table(a, d, mark, n) :

    for i in range(n) :
        # очистить метки
        clean_mark(mark, n)

        # волна со стартовым атомом i
        wave(a, i, n, mark)

        # заполнить i-тую строку матрицы расстояний
        init_distance_table_i(d, mark, i, n)

In [50]:
def init_deg_array (a, deg, n) :
    
    for i in range(n) :
        sum = 0
        for j in range(n) :
            if a[i][j] == 1 :
                sum += 1
        deg[i] = sum

In [51]:
def calc_Wiener_index(d, n) :
    
    sum = 0
    
    for i in range(n) :
        for j in range(n) :
            sum += d[i][j]
            
    return sum / 2

In [52]:
def calc_Randich_index(d, deg, n) :
  
    sum = 0

    for i in range(n) :
        for j in range(n) :
            if d[i][j] == 1 :
                sum += 1 / np.sqrt(deg[i] * deg[j])
                
    return (sum / 2)

In [53]:
def find_diam_and_rad (d, n) :

    loc_max = np.zeros(n, dtype = int, order = 'C')

    for i in range(n) :
        max = 0

        for j in range(n) :
            if d[i][j] > max:
                max = d[i][j]

        loc_max[i] = max

    rad = loc_max[0]
    diam = loc_max[0]

    for i in range(n) :
        if loc_max[i] < rad :
            rad = loc_max[i]

        if loc_max[i] > diam :
            diam = loc_max[i]
            
    return diam, rad

In [54]:
diam = np.zeros(len(ms), dtype = int, order = 'C')
rad = np.zeros(len(ms), dtype = int, order = 'C')
Wiener_indices = np.zeros(len(ms), dtype = int, order = 'C')
Randich_indices = np.zeros(len(ms), dtype = float, order = 'C')

for i in range(len(ms)) :
    n = ms[i].GetNumAtoms()
    a = Chem.rdmolops.GetAdjacencyMatrix(ms[i], useBO=False, emptyVal=0, force=False, prefix='')
     
    d = np.zeros((n, n), dtype = int, order = 'C')
    
    mark = np.zeros(n, dtype = int, order = 'C')
    deg = np.zeros(n, dtype=int, order='C')
    
    init_distance_table(a, d, mark, n)
    init_deg_array (a, deg, n)
    
    diam[i], rad[i] = find_diam_and_rad(d, n)
    Wiener_indices[i] = calc_Wiener_index(d, n)
    Randich_indices[i] = calc_Randich_index(d, deg, n)

In [55]:
np.savetxt('cox2-diam_rad_Wiener_Randich.csv', [p for p in zip(diam, rad, Wiener_indices, Randich_indices)], delimiter=',', fmt='%.3f')